# Calculate monthly mean data from daily inputs

## notes

## imports

In [17]:
%%time
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os
import xarray as xr
import xcdat as xc

CPU times: user 18 µs, sys: 8 µs, total: 26 µs
Wall time: 29.6 µs


## functions

In [18]:
def nearestNeighbourFill(data, missingValue=0):
    """
    Documentation for nearestNeighbourFill():
    -------
    The nearestNeighbourFill() function iteratively infills a 2D matrix
    with values from immediately neighbouring cells

    Author: Paul J. Durack : pauldurack@llnl.gov

    Inputs:
    -----

    |  **data** - a numpy 2D array
    |  **missingValue** - missing value of data matrix

    Returns:
    -------

    |  **filledData** - a numpy array with no missingValues

    Usage:
    ------
        data = np.array([[1, 2, 3, 4],
                         [5, 0, 7, 8],
                         [9, 10, 11, 12]])

        filledData = nearestNeighborFill(data, missingValue=0)
        print(filledData)
    
    Notes:
    -----
    * PJD 28 Nov 2023 - Started
    """

    # Make copy of input matrix
    filledData = data.copy()

    # Find indices of missing values
    missingIndices = np.argwhere(data == missingValue)

    for idx in missingIndices:
        row, col = idx
        neighbors = []

        # Iterate over neighbouring cells
        for i in range(max(0, row - 1), min(data.shape[0], row + 2)):
            for j in range(max(0, col - 1), min(data.shape[1], col + 2)):
                if (i, j) != (row, col) and data[i, j] != missingValue:
                    neighbours.append(data[i, j])

        # Fill missing value with the mean of neighbours
        if neighbours:
            filledData[row, col] = np.mean(neighbours)

    return filledData


def iterativeZonalFill(data, missingValue=0):
    """
    Documentation for iterativeZonalFill():
    -------
    The iterativeZonalFill() function iteratively infills a 2D matrix
    with values zonal neighbouring cells

    Author: Paul J. Durack : pauldurack@llnl.gov

    Inputs:
    -----

    |  **data** - a numpy 2D array
    |  **missingValue** - missing value of data matrix

    Returns:
    -------

    |  **filledData** - a numpy array with no missingValues

    Usage:
    ------
        data = np.array([[1, 2, 3, 4],
                         [5, 0, 7, 8],
                         [9, 10, 11, 12]])

        filledData = iterativeZonalFill(data, missingValue=0)
        print(filledData)
    
    Notes:
    -----
    * PJD 28 Nov 2023 - Started
    """
   
    # Make copy of input matrix
    filledData = data.copy()

    # Find indices of missing values
    missingIndices = np.argwhere(data == missingValue)

    # Define directions for iteration (right, down, left, up)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    for direction in directions:
        dx, dy = direction

        # Iterate over the data in the specified direction
        for i in range(1, max(data.shape) + 1):
            for idx in missingIndices:
                row, col = idx
                new_row, new_col = row + i * dx, col + i * dy

                # Check if the new indices are within the data boundaries
                if 0 <= new_row < data.shape[0] and 0 <= new_col < data.shape[1]:
                    if data[new_row, new_col] != missingValue:
                        filledData[row, col] = data[new_row, new_col]

    return filledData



## set data paths

In [19]:
obsPath = "/p/user_pub/PCMDIobs/obs4MIPs_input/RSS/RSS-MW5-1/v20230605/"

## lazy load data

In [56]:
def setCalendar(ds):
    # https://github.com/pydata/xarray/issues/6259
    ds.time.attrs["calendar"] = "standard"
    ds.time.attrs["units"] = "seconds since 1981-01-01 00:00:00"
    return ds
    #return xr.decode_cf(ds)

#dataPath = os.path.join(obsPath, "199801*.nc")
dataPath = os.path.join(obsPath, "1998*.nc")
print("dataPath:", dataPath)
ds = xc.open_mfdataset(dataPath, preprocess=setCalendar)
print("done!")

dataPath: /p/user_pub/PCMDIobs/obs4MIPs_input/RSS/RSS-MW5-1/v20230605/1998*.nc
done!


## view dataset

In [57]:
ds

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 365, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) object 1998-01-01 12:00:00 ... 1998-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lon_bnds          (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 180.0
    lat_bnds          (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 90.0
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

## add calendar attribute to time axis

In [37]:
ds.time.attrs['calendar'] = 'standard'

In [38]:
ds

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 365, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) object 1998-01-01 12:00:00 ... 1998-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lon_bnds          (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 180.0
    lat_bnds          (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 90.0
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

## add missing time_bnds

In [39]:
ds = ds.bounds.add_missing_bounds(axes="T")
#ds.time.attrs["bounds"] = "time_bnds"

In [40]:
ds

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 365, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) object 1998-01-01 12:00:00 ... 1998-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lon_bnds          (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 180.0
    lat_bnds          (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 90.0
    time_bnds         (time, bnds) object 1998-01-01 00:00:00 ... 1999-01-01 ...
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [41]:
mean_monthXr = ds.analysed_sst.resample(time='M').mean()
# https://stackoverflow.com/questions/50564459/using-xarray-to-make-monthly-average

In [43]:
mean_monthXr

<xarray.DataArray 'analysed_sst' (time: 12, lat: 720, lon: 1440)>
dask.array<stack, shape=(12, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) object 1998-01-31 00:00:00 ... 1998-12-31 00:00:00
Attributes:
    units:                  K
    long_name:              analysed sea surface temperature
    standard_name:          sea_surface_foundation_temperature
    valid_min:              -32767
    valid_max:              32767
    source:                 REMSS-L3C-TMI
    coverage_content_type:  physicalMeasurement

In [48]:
mean_monthXc2 = mean_monthXc.bounds.add_time_bounds(method="freq", freq="month")

In [49]:
mean_monthXc2

<xarray.Dataset>
Dimensions:       (lat: 720, lon: 1440, bnds: 2, time: 12)
Coordinates:
  * lat           (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) object 1998-01-01 00:00:00 ... 1998-12-01 00:00:00
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 179.8 180.0
    lat_bnds      (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 89.75 90.0
    analysed_sst  (time, lat, lon) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    time_bnds     (time, bnds) object 1998-01-01 00:00:00 ... 1999-01-01 00:0...
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [50]:
mean_monthXc3 = mean_monthXc2.temporal.group_average("analysed_sst", freq="month", weighted=True)

In [51]:
mean_monthXc3

<xarray.Dataset>
Dimensions:       (lat: 720, lon: 1440, bnds: 2, time: 12)
Coordinates:
  * lat           (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) object 1998-01-01 00:00:00 ... 1998-12-01 00:00:00
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 179.8 180.0
    lat_bnds      (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 89.75 90.0
    analysed_sst  (time, lat, lon) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [45]:
mean_monthXc2 = ds.temporal.average("analysed_sst", weighted=True)

In [46]:
mean_monthXc2

<xarray.Dataset>
Dimensions:       (lat: 720, lon: 1440, bnds: 2)
Coordinates:
  * lat           (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 179.8 180.0
    lat_bnds      (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 89.75 90.0
    analysed_sst  (lat, lon) float64 dask.array<chunksize=(720, 1440), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [ ]:
mean_monthXc.time

In [30]:
mean_monthXc2 = mean_monthXc.bounds.add_missing_bounds(axes="T")

2023-11-29 10:21:47,798 [WARNING]: bounds.py(add_missing_bounds:186) >> Cannot generate bounds for coordinate variable 'time' which has a length <= 1 (singleton).
2023-11-29 10:21:47,798 [WARNING]: bounds.py(add_missing_bounds:186) >> Cannot generate bounds for coordinate variable 'time' which has a length <= 1 (singleton).


In [31]:
mean_monthXc2

<xarray.Dataset>
Dimensions:       (lat: 720, lon: 1440, bnds: 2, time: 1)
Coordinates:
  * lat           (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) object 1998-01-01 00:00:00
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 179.8 180.0
    lat_bnds      (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 89.75 90.0
    analysed_sst  (time, lat, lon) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [12]:
ds.analysed_sst

<xarray.DataArray 'analysed_sst' (time: 31, lat: 720, lon: 1440)>
dask.array<concatenate, shape=(31, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) object 1998-01-01 12:00:00 ... 1998-01-31 12:00:00
Attributes:
    units:                  K
    long_name:              analysed sea surface temperature
    standard_name:          sea_surface_foundation_temperature
    valid_min:              -32767
    valid_max:              32767
    source:                 REMSS-L3C-TMI
    coverage_content_type:  physicalMeasurement